# Final Project Notebook

The report for this final project can be found at this [link](https://cybertraining-dsc.github.io/report/fa20-523-301/project/project/).

## Part 1 Importing the functions

This file requires that we import Numpy, Matplotlib, Pylab, Keras, and Pandas

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
import os, sys
import pandas as pd
import io
import requests
import warnings

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
! pip install cloudmesh-common -U

from cloudmesh.common.Benchmark import Benchmark

     |████████████████████████████████| 81kB 2.3MB/s 
     |████████████████████████████████| 133kB 8.4MB/s 
  Created wheel for python-hostlist: filename=python_hostlist-1.21-cp36-none-any.whl size=38932 sha256=609788a22cddbada20ee24874009b837b448033df9cf7149f4ca6f8cde136d2b
  Stored in directory: /root/.cache/pip/wheels/0b/5b/55/ddcf52288f0b10f4564ca1b2531594ff7ccc65f487ba8dc437
Successfully built python-hostlist


In [3]:
! pip install utils
import utils

Now that the funtions have been imported the team can focus on the download coding. The following cells will set up an install for Kaggle files and prompt for an upload of the kaggle.json file for credentials. 

The mkdir function creates a directory for the Kaggle data. This cell will allow the team to verify that the kaggle.json file appropriately uploaded to the directory.

In [ ]:
##import the kaggle.json from local to drive
!pip install -q kaggle
from google.colab import files
##when it asks you to choose a file select the kaggle.json located within the 'project' folder from the github repo
files.upload()

In [ ]:
##make a kaggle and a data folder
#!mkdir ~/.kaggle
!mkdir data
##copy the kaggle.json to the .kaggle folder then grant permissions
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
#test to see if kaggle is working, should print list of datasets
!kaggle datasets list

mkdir: cannot create directory ‘data’: File exists
ref                                                               title                                                 size  lastUpdated          downloadCount  
----------------------------------------------------------------  --------------------------------------------------  ------  -------------------  -------------  
nehaprabhavalkar/indian-food-101                                  Indian Food 101                                        7KB  2020-09-30 06:23:43           4562  
christianlillelund/donald-trumps-rallies                          Donald Trump's Rallies                               720KB  2020-09-26 10:25:08           1141  
heeraldedhia/groceries-dataset                                    Groceries dataset                                    257KB  2020-09-17 04:36:08           5102  
andrewmvd/trip-advisor-hotel-reviews                              Trip Advisor Hotel Reviews                             5MB  2020-09-

Now, the team must download all of the datasets for the class. The three datasets are focused on the NBA. 

The first dataset is for injuries. Each injury will be used to set up players, timeframes, and severity of injuries. 

The other two datasets are for the player performance. By cross referencing this data to the previous list, the team will be able to see which players are limited from the injury and how performance is hampered by time in rehab.

In [ ]:
##downloading all the datasets
!kaggle datasets download -d ghopkins/nba-injuries-2010-2018
!kaggle datasets download -d nathanlauga/nba-games
!kaggle datasets download -d pablote/nba-enhanced-stats

nba-injuries-2010-2018.zip: Skipping, found more recently modified local copy (use --force to force download)
nba-games.zip: Skipping, found more recently modified local copy (use --force to force download)
nba-enhanced-stats.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
##unzipping to the data folder
!unzip nba-injuries-2010-2018.zip -d data
!unzip nba-games.zip -d data
!unzip nba-enhanced-stats.zip -d data

Archive:  nba-injuries-2010-2018.zip
  inflating: data/injuries_2010-2020.csv  
Archive:  nba-games.zip
  inflating: data/games.csv          
  inflating: data/games_details.csv  
  inflating: data/players.csv        
  inflating: data/ranking.csv        
  inflating: data/teams.csv          
Archive:  nba-enhanced-stats.zip
  inflating: data/2012-18_officialBoxScore.csv  
  inflating: data/2012-18_playerBoxScore.csv  
  inflating: data/2012-18_standings.csv  
  inflating: data/2012-18_teamBoxScore.csv  
  inflating: data/2016-17_officialBoxScore.csv  
  inflating: data/2016-17_playerBoxScore.csv  
  inflating: data/2016-17_standings.csv  
  inflating: data/2016-17_teamBoxScore.csv  
  inflating: data/2017-18_officialBoxScore.csv  
  inflating: data/2017-18_playerBoxScore.csv  
  inflating: data/2017-18_standings.csv  
  inflating: data/2017-18_teamBoxScore.csv  
  inflating: data/metadata_officialBoxScore.pdf  
  inflating: data/metadata_playerBoxScore.pdf  
  inflating: data/metadata

The team must now use these downloads to create dataframes. Pandas dataframes will be easier to manage the data. The team will be able to use Pandas to process the data and allow the team to make correlations for feature engineering to create the models.

In [ ]:
#create a list for each data set
ds_NBA_Injuries, ds_NBA_Games, ds_NBA_Enhanced = [], [], []

#import csv files as dataframes and save to respective list, injury set first
df_Injuries = pd.read_csv('data/injuries_2010-2020.csv')
df_Injury_Start = df_Injuries[df_Injuries.Acquired.isnull()]
df_Injury_End = df_Injuries[df_Injuries.Relinquished.isnull()]
ds_NBA_Injuries = [df_Injury_Start, df_Injury_End]
#nba games dataset
df_Games_games = pd.read_csv('data/games.csv')
df_Games_gamesDetails = pd.read_csv('data/games_details.csv')
df_Games_players = pd.read_csv('data/players.csv')
df_Games_ranking = pd.read_csv('data/ranking.csv')
df_Games_teams = pd.read_csv('data/teams.csv')
ds_NBA_Games = [df_Games_games, df_Games_gamesDetails, df_Games_players, df_Games_ranking, df_Games_teams]
#nba enhanced stats dataset
df_En_officialBS_1218 = pd.read_csv('data/2012-18_officialBoxScore.csv')
df_En_playerBS_1218 = pd.read_csv('data/2012-18_playerBoxScore.csv')
df_En_standings_1218 = pd.read_csv('data/2012-18_standings.csv')
df_En_teamBS_1218 = pd.read_csv('data/2012-18_teamBoxScore.csv')  
df_En_officialBS_1617 = pd.read_csv('data/2016-17_officialBoxScore.csv')  
df_En_playerBS_1617 = pd.read_csv('data/2016-17_playerBoxScore.csv')
df_En_standings_1617 = pd.read_csv('data/2016-17_standings.csv')
df_En_teamBS_1617 = pd.read_csv('data/2016-17_teamBoxScore.csv')  
df_En_officialBS_1718 = pd.read_csv('data/2017-18_officialBoxScore.csv')  
df_En_playerBS_1718 = pd.read_csv('data/2017-18_playerBoxScore.csv')
df_En_standings_1718 = pd.read_csv('data/2017-18_standings.csv')
df_En_teamBS_1718 = pd.read_csv('data/2017-18_teamBoxScore.csv')  
##data/metadata_officialBoxScore.pdf, data/metadata_playerBoxScore.pdf, data/metadata_standing.pdf, data/metadata_teamBoxScore.pdf  
df_En_teamBS = pd.read_csv('data/teamBoxScore.csv')
ds_NBA_Enhanced = [df_En_officialBS_1218, df_En_officialBS_1617, df_En_officialBS_1718, df_En_playerBS_1218, df_En_playerBS_1617, df_En_playerBS_1718, df_En_standings_1218, df_En_standings_1617, df_En_standings_1718, \
                       df_En_teamBS_1218, df_En_teamBS_1617, df_En_teamBS_1718, df_En_teamBS]


#probably need some more data exploration and some feature engineering

## Exploratory Data Analysis

At this point, it is time to build into new useful sets of data. The team will explore different sets to combine in to the models to be trained.

In [ ]:
df_distinct_playerID = df_Games_players[["PLAYER_NAME", "PLAYER_ID"]].drop_duplicates()
df_distinct_playerID.astype({'PLAYER_ID':'object'}).dtypes



# df_distinct_playerID=df_distinct_playerID.sort_values('PLAYER_NAME')
# df_distinct_playerID = df_distinct_playerID.sort_values(by=['PLAYER_NAME']).reset_index(drop=True, inplace=True)
#df_Injury_End

PLAYER_NAME    object
PLAYER_ID      object
dtype: object

In [ ]:
df_distinct_playerID

,PLAYER_NAME,PLAYER_ID
0,Royce O'Neale,1626220
1,Bojan Bogdanovic,202711
2,Rudy Gobert,203497
3,Donovan Mitchell,1628378
4,Mike Conley,201144
...,...,...
7223,Lanny Smith,201831
7224,Warren Carter,201999
7225,Bennet Davis,201834
7226,Brian Hamilton,201646


In [ ]:
#join player ID, 
df_Injury_Start = df_Injury_Start.join(df_distinct_playerID.astype('object').set_index('PLAYER_NAME'), on='Relinquished')


ValueError: ignored

,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...,2430
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely),201973
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...,202358
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely),201965
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely),2211
...,...,...,...,...,...,...
27097,2020-09-22,Celtics,NaN,Romeo Langford,surgery on right wrist (out for season),1629641
27098,2020-09-23,Heat,NaN,Gabe Vincent,sore right knee (DTD),NaN
27099,2020-09-30,Heat,NaN,Bam Adebayo,strained left shoulder (DTD),1628389
27101,2020-10-02,Heat,NaN,Bam Adebayo,strained neck (DTD),1628389


,PLAYER_NAME,PLAYER_ID
0,Royce O'Neale,1626220
1,Bojan Bogdanovic,202711
2,Rudy Gobert,203497
3,Donovan Mitchell,1628378
4,Mike Conley,201144
...,...,...
7223,Lanny Smith,201831
7224,Warren Carter,201999
7225,Bennet Davis,201834
7226,Brian Hamilton,201646


# Part 2 Building the Keras Model

A link for Keras for us to use can be found [here](https://keras.io/guides/sequential_model/). We are first going to set up our Benchmark Test to be used when we are Benchmarking our model.

In [ ]:
def b():
  Benchmark.Start()
  print ("b")
  import time
  time.sleep(3)
  Benchmark.Stop()

def c():
  Benchmark.Start()
  print ("c")
  import time
  time.sleep(1)
  Benchmark.Stop()

In [ ]:
 b()
 c()

 Benchmark.print()

b
c

+---------------------+------------------------------------------------------------------+
| Attribute           | Value                                                            |
|---------------------+------------------------------------------------------------------|
| BUG_REPORT_URL      | "https://bugs.launchpad.net/ubuntu/"                             |
| DISTRIB_CODENAME    | bionic                                                           |
| DISTRIB_DESCRIPTION | "Ubuntu 18.04.5 LTS"                                             |
| DISTRIB_ID          | Ubuntu                                                           |
| DISTRIB_RELEASE     | 18.04                                                            |
| HOME_URL            | "https://www.ubuntu.com/"                                        |
| ID                  | ubuntu                                                           |
| ID_LIKE             | debian                                                       

Now that we know which GPU we are using, we can get into the actual work. The following is building our Keras model.

#TO ADD

In [ ]:
import warnings

np.random.seed(23)
warnings.filterwarnings("ignore")

In [ ]:
df_baseline = df_Injury_Start

#df_baseline.sort_values(by=['Date','Name']).reset_index(drop=True, inplace=True)
df_baseline.sort_values(by=['Relinquished']).reset_index(drop=True, inplace=True)
# df_baseline['FPTS_pred'] = utils.calculate_FPTS(df_baseline)

# # Season average
# print(' MAE | ', utils.calculate_MAE(df_baseline['FPTS_pred'], df_baseline['FPTS']))
# print('RMSE | ', utils.calculate_RMSE(df_baseline['FPTS_pred'], df_baseline['FPTS']))

In [ ]:
df_baseline

,Date,Team,Acquired,Relinquished,Notes,PLAYER_ID
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...,2430
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely),201973
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...,202358
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely),201965
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely),2211
...,...,...,...,...,...,...
27097,2020-09-22,Celtics,NaN,Romeo Langford,surgery on right wrist (out for season),1629641
27098,2020-09-23,Heat,NaN,Gabe Vincent,sore right knee (DTD),NaN
27099,2020-09-30,Heat,NaN,Bam Adebayo,strained left shoulder (DTD),1628389
27101,2020-10-02,Heat,NaN,Bam Adebayo,strained neck (DTD),1628389


In [ ]:
#df_baseline.sort_values(by=['Date','Name']).reset_index(drop=True, inplace=True)
df_baseline.sort_values(by=['Relinquished']).reset_index(drop=True, inplace=True)


# df_baseline['	PLAYER_ID'] = utils.calculate_FPTS(df_baseline)

# # Season average
# print(' MAE | ', utils.calculate_MAE(df_baseline['FPTS_pred'], df_baseline['FPTS']))
# print('RMSE | ', utils.calculate_RMSE(df_baseline['FPTS_pred'], df_baseline['FPTS']))

In [ ]:
df_baseline = df_baseline
# basic =  ['PTS','3P','AST','TRB','STL','BLK','TOV', 'DD', 'TD']

X = df_baseline.loc['PLAYER_ID']
# X = df_baseline.loc[:, basic]
X = MinMaxScaler().fit_transform(X)
y = df_baseline['FPTS'].values.reshape(-1,1).flatten()

reg = LinearRegression()
errors = utils.cross_val(reg, X, y, n_folds=5, verbose=0)
utils.summarize_errors(errors)

KeyError: ignored

In [ ]:
# When the dataframes are combined, use this code to select features.

features = ['SG', 'F', 'C', 'PTS', '3P', 'AST', 'TRB', 'STL', 'BLK', 'TOV', 'DD', 'TD', 'MP', 'FT',
            'FTA', 'FGA', '3PA', 'DRB', 'ORB', 'USG_perc', 'DRtg', 'ORtg', 'AST_perc', 'DRB_perc',
            'ORB_perc', 'BLK_perc', 'TOV_perc', 'STL_perc', 'eFG_perc', 'FG_perc', '3P_perc', 'FT_perc']

In [ ]:
for weighting in ['sqrt', 'linear', 'quad']:
    print("\nweighting sheme:", weighting)
    df_features = utils.csv_concatenate(os.path.join(DATA_DIR, 'Dataframes','Modelling', 'Features', weighting))  
    
    X = df_features.loc[:, features]
    X = MinMaxScaler().fit_transform(X)
    y = df_features['FPTS'].values.reshape(-1,1).flatten()

    reg = LinearRegression()

#Utils isn't working for me. We're going to have to find a new Import (Unless you know what I'm doing wrong with utils)

    errors = utils.cross_val(reg, X, y, n_folds=5, verbose=0)
    utils.summarize_errors(errors, verbose=0)

In [ ]:
weighting = 'quad'
df_features = utils.csv_concatenate(os.path.join(DATA_DIR, 'Dataframes','Modelling', 'Features', weighting))

In [ ]:
_all = ['Salary', 'Rest', 'Rota_All', 'Rota_Pos', 'Home', 'SG', 'F', 'C', 'Value', 'FPTS_std',
        'PTS', '3P', 'AST', 'TRB', 'STL', 'BLK', 'TOV', 'DD', 'TD', 'MP', 'FT', 'FTA', 'FGA', '3PA', 'DRB',
        'ORB', 'USG_perc', 'DRtg', 'ORtg', 'AST_perc', 'DRB_perc', 'ORB_perc', 'BLK_perc', 'TOV_perc', 
        'STL_perc', 'eFG_perc', 'FG_perc', '3P_perc', 'FT_perc']

In [ ]:

X = df_features.loc[:, _all]
X = MinMaxScaler().fit_transform(X)
y = df_features['FPTS'].values.reshape(-1,1).flatten()

# Takes 2 minutes
model = GradientBoostingRegressor()
model.fit(X, y)

top_features = pd.Series(model.feature_importances_, index = _all).sort_values()
top_features.plot(kind = "barh", figsize=(15,10) ,title='Top Features')
plt.show()

In [ ]:
omit_lowest = 20
_selected = list(top_features[omit_lowest:].index)

In [ ]:
for feature_type in [_all, _selected]:
    for weighting in ['sqrt', 'linear', 'quad']:
        df_features = utils.csv_concatenate(os.path.join(DATA_DIR, 'Dataframes','Modelling',
                                                         'Features', weighting))
        print("\nweighting sheme: {}".format(weighting))

        X = df_features.loc[:, feature_type]
        X = MinMaxScaler().fit_transform(X)
        y = df_features['FPTS'].values.reshape(-1,1).flatten()
        
        reg = LinearRegression()

        errors = utils.cross_val(reg, X, y, verbose=0)
        utils.summarize_errors(errors, verbose=0)
        
    print("========================")

# DONE ADDING

In [ ]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)


x = df_baseline
# x = tf.ones((3, 3))
y = model(x)

Insert Layers

In [ ]:
# Create 3 layers
layer1 = layers.Dense(2, activation="relu", name="layer1")
layer2 = layers.Dense(3, activation="relu", name="layer2")
layer3 = layers.Dense(4, name="layer3")

# Call layers on a test input
x = df_baseline
y = layer3(layer2(layer1(x)))

# Part 3 Conclusions

This is where the conclusions section will be typed